In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recsys-yandex-cup-2022/track_artists.csv
/kaggle/input/recsys-yandex-cup-2022/test
/kaggle/input/recsys-yandex-cup-2022/train
/kaggle/input/recsys-yandex-cup-2022/dataframe.csv/dataframe.csv


# Явно делим данные

Домашнее задание №2.1

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

Итоговая задача - построить схему валидации для данного соревнования с учетом особенностей сорвенования

Между train и test не должно быть общих пользователей
Количество фолдов задается через параметр класса n_folds
В test должно быть не более p последних треков (параметр класса p)

In [2]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])
df = pd.read_csv('/kaggle/input/recsys-yandex-cup-2022/dataframe.csv/dataframe.csv') 
headtail(df)

,user_id,track_id,rank,reversed_rank,is_train
0,0,333396,0,53,1
1,0,267089,1,52,1
2,0,155959,2,51,1
3,0,353335,3,50,1
4,0,414000,4,49,1
117450829,289913,448288,34,4,0
117450830,289913,1343,35,3,0
117450831,289913,86420,36,2,0
117450832,289913,186436,37,1,0
117450833,289913,8474,38,0,0


In [3]:
max_train_user_id = df[df.is_train == 1].user_id.max()
max_train_user_id
max_test_user_id = df[df.is_train == 0].user_id.max()
max_train_user_id

1160083

In [4]:
df.loc[df["is_train"] == 0, "user_id"] +=  max_train_user_id
df

,user_id,track_id,rank,reversed_rank,is_train
0,0,333396,0,53,1
1,0,267089,1,52,1
2,0,155959,2,51,1
3,0,353335,3,50,1
4,0,414000,4,49,1
...,...,...,...,...,...
117450829,1449996,448288,34,4,0
117450830,1449996,1343,35,3,0
117450831,1449996,86420,36,2,0
117450832,1449996,186436,37,1,0


In [5]:
df = df[["user_id", "track_id", "reversed_rank"]]
df = df.rename(columns={"user_id": "user", "track_id": "item", "reversed_rank": "order"})
df

,user,item,order
0,0,333396,53
1,0,267089,52
2,0,155959,51
3,0,353335,50
4,0,414000,49
...,...,...,...
117450829,1449996,448288,4
117450830,1449996,1343,3
117450831,1449996,86420,2
117450832,1449996,186436,1



# Класс, реализующий схему валидации UsersKFoldLeavePOut

Решаемая задача - построить схему валидации для данного соревнования с учетом особенностей сорвенования

    Между train и test не должно быть общих пользователей
    Количество фолдов задается через параметр класса n_folds
    В test должно быть не более p последних треков (параметр класса p)

In [6]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df):
                # Split method that takes in a DataFrame to perform user-based k-fold cross validation.
        users = df.user.unique()  # Get unique users from the user column in the DataFrame.
        users_count = len(users)  # Count the number of unique users.
        
        np.random.seed(self.random_seed)  # Set random seed for reproducibility.
        np.random.shuffle(users)  # Shuffle the users randomly.
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)  # Compute fold sizes based on number of users and desired number of folds.
        fold_sizes[: users_count % self.n_folds] += 1  # Distribute any remaining users to the first few folds.
        current = 0  # Initialize current pointer for iterating over fold sizes.
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size  # Determine the start and stop index for the current fold.
            test_fold_users = users[start:stop]  # Get users for the current fold.
            # В test должно быть не более p последних треков (параметр класса p) по каждому пользователю
            test_mask = (df.user.isin(test_fold_users)) & (df["order"] < self.p)
            # test_mask = df[self.user_column].isin(test_fold_users)  # Create a boolean mask for test set users in the DataFrame.
            train_mask = ~test_mask.isin(test_fold_users) # Берем всех остальных юзеров, не включая оценки юзеров, которые были в тестовом
            # train_mask = ~test_mask  # Invert the test mask to get the train set users.
            
            yield train_mask, test_mask
        
        

In [7]:
cv = UsersKFoldPOut(n_folds=3, p=1)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
     train = df[train_mask]
     test = df[test_mask]
     print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 116967501, Test: 483333
Fold#1 | Train: 116967502, Test: 483332
Fold#2 | Train: 116967502, Test: 483332
